In [1]:
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
import sec_parser as sp
from sec_downloader import Downloader
import warnings
from sec_parser import Edgar10QParser, TreeBuilder, render
from sec_parser.processing_steps import TextClassifier


In [2]:
def get_random_headers():
    ua = UserAgent()
    headers = {'User-Agent': ua.random}
    return headers

In [3]:


# URL to the EDGAR filing
url = "https://www.sec.gov/Archives/edgar/data/102909/0001104659-20-018237.txt"

# Send a GET request to the URL
response = requests.get(url, headers=get_random_headers())

# Check if the request was successful
if response.status_code == 200:
    content = response.text
    print(content)  # or process the content as needed
else:
    print("Failed to retrieve the document")


<SEC-DOCUMENT>0001104659-20-018237.txt : 20200212
<SEC-HEADER>0001104659-20-018237.hdr.sgml : 20200212
<ACCEPTANCE-DATETIME>20200212092349
ACCESSION NUMBER:		0001104659-20-018237
CONFORMED SUBMISSION TYPE:	SC 13G/A
PUBLIC DOCUMENT COUNT:		1
FILED AS OF DATE:		20200212
DATE AS OF CHANGE:		20200212

SUBJECT COMPANY:	

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			HOME BANCSHARES INC
		CENTRAL INDEX KEY:			0001331520
		STANDARD INDUSTRIAL CLASSIFICATION:	STATE COMMERCIAL BANKS [6022]
		IRS NUMBER:				710682831
		STATE OF INCORPORATION:			AR
		FISCAL YEAR END:			1231

	FILING VALUES:
		FORM TYPE:		SC 13G/A
		SEC ACT:		1934 Act
		SEC FILE NUMBER:	005-83723
		FILM NUMBER:		20600474

	BUSINESS ADDRESS:	
		STREET 1:		719 HARKRIDER
		CITY:			CONWAY
		STATE:			AR
		ZIP:			72032
		BUSINESS PHONE:		501-339-2929

	MAIL ADDRESS:	
		STREET 1:		719 HARKRIDER
		CITY:			CONWAY
		STATE:			AR
		ZIP:			72032

FILED BY:		

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			VANGUARD GROUP INC
		CENTRAL INDEX KEY:			0

In [4]:
url = "https://www.sec.gov/Archives/edgar/data/1015383/0001493152-20-002565.txt"

# Send a GET request to the URL
response = requests.get(url, headers=get_random_headers())

response.raise_for_status()  # Ensure the request was successful

# Create a BeautifulSoup object
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the starting point
    start_anchor = soup.find('a', attrs={'name': 'gg_002'})
    print(start_anchor)
    # Find the endpoint
    end_anchor = soup.find('a', attrs={'name': 'gg_003'})
    print(end_anchor)

    # Variable to hold all relevant paragraphs
    content = []

    if start_anchor and end_anchor:
        # Start collecting the content from the next sibling of start_anchor
        element = start_anchor.find_next_sibling()
        print(element)

        while element and element != end_anchor:
            print(element)
            if element.name == 'p':
                content.append(element.get_text(strip=True))
            element = element.find_next_sibling()

        # Join the paragraphs with new lines to form the complete section
        full_text = "\n".join(content)
        print(full_text)
    else:
        print("Start or end anchor not found.")

else:
    print("Failed to retrieve the document")

<a name="gg_002"></a>
<a name="gg_003"></a>
<b>ITEM
2. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS.</b>
<b>ITEM
2. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS.</b>



In [6]:
url = "https://www.sec.gov/Archives/edgar/data/1015383/0001493152-20-002565.txt"



elements: list = sp.Edgar10QParser().parse(url)
print(list)


demo_output: str = sp.render(elements)
print(demo_output)


<class 'list'>
TextElement: https://www.sec.gov/Archives/ed...015383/0001493152-20-002565.txt


c:\Users\310\Desktop\Progects_Py\env\Lib\site-packages\sec_parser\processing_engine\html_tag_parser.py:50: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  root: bs4.Tag = bs4.BeautifulSoup(


In [27]:
dl = Downloader("MyCompanyName", "email@example.com")
html = dl.get_filing_html(ticker="AAPL", form="10-Q")
print(html[:100])



b'<?xml version=\'1.0\' encoding=\'ASCII\'?>\n<html xmlns:ecd="http://xbrl.sec.gov/ecd/2023" xmlns:ix="http'


In [32]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message="Invalid section type for")
    elements: list = sp.Edgar10QParser().parse(html)

print(elements)
title_elements = [element for element in elements if 'TitleElement' in type(element).__name__]


[IntroductorySectionElement<table>, TopSectionTitle[L0]<span>, TopSectionTitle[L1]<span>, TitleElement[L0]<div>, SupplementaryText<div>, TableElement<div>, SupplementaryText<div>, TitleElement[L0]<div>, SupplementaryText<div>, TableElement<div>, SupplementaryText<div>, TitleElement[L0]<div>, SupplementaryText<div>, TableElement<div>, SupplementaryText<div>, TitleElement[L0]<div>, SupplementaryText<div>, TableElement<div>, SupplementaryText<div>, TitleElement[L0]<div>, SupplementaryText<div>, TableElement<div>, SupplementaryText<div>, TitleElement[L1]<div>, TitleElement[L2]<div>, TitleElement[L2]<div>, TextElement<div>, TextElement<div>, TitleElement[L2]<div>, TextElement<div>, TableElement<div>, TextElement<sec-parser-merged-text>, TitleElement[L2]<div>, TextElement<div>, TableElement<div>, TextElement<div>, TitleElement[L2]<div>, TitleElement[L2]<div>, TextElement<div>, TableElement<div>, TableElement<div>, TextElement<sec-parser-merged-text>, TextElement<div>, TableElement<div>, Titl

In [33]:
demo_output: str = sp.render(elements)
#demo_output: str = render_full_text(title_elements)
print(demo_output)

TopSectionTitle: PART I  —  FINANCIAL INFORMATION
TopSectionTitle: Item 1.    Financial Statements
TitleElement: CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS (Unaudited)
SupplementaryText: (In millions, except number of ...ousands, and per-share amounts)
TableElement: Table with ~24 rows, ~40 numbers, and 742 characters.
SupplementaryText: See accompanying Notes to Conde...solidated Financial Statements.
TitleElement: CONDENSED CONSOLIDATED STATEMEN...OMPREHENSIVE INCOME (Unaudited)
SupplementaryText: (In millions)
TableElement: Table with ~13 rows, ~22 numbers, and 839 characters.
SupplementaryText: See accompanying Notes to Conde...solidated Financial Statements.
TitleElement: CONDENSED CONSOLIDATED BALANCE SHEETS (Unaudited)
SupplementaryText: (In millions, except number of ...ed in thousands, and par value)
TableElement: Table with ~35 rows, ~57 numbers, and 1358 characters.
SupplementaryText: See accompanying Notes to Conde...solidated Financial Statements.
TitleElement: CONDEN

In [ ]:
tree = sp.TreeBuilder().build(elements)

demo_output: str = sp.render(tree)
print(demo_output)

In [22]:
def render_full_text(elements):
    full_text = ""
    for element in elements:
        if hasattr(element, "text"):
            full_text += element.text + "\n"
        # You may need to handle other types of elements if necessary
    return full_text

In [86]:
# Define a custom classifier that processes all text elements fully
class FullTextClassifier(TextClassifier):
    def _process_element(self, element, context):
        # Assume all text elements contain full text; avoid truncating
        return element  # Return the element unchanged

# Replace only the text classification step with the full text classifier
def get_full_text_steps():
    return [
        FullTextClassifier() if isinstance(step, TextClassifier) else step
        for step in Edgar10QParser().get_default_steps()
    ]

# Parse the document using the adjusted steps
parser = Edgar10QParser(get_full_text_steps)
elements = parser.parse(html)

demo_output: str = sp.render(elements)
print(demo_output)

TopSectionTitle: PART I  —  FINANCIAL INFORMATION
TopSectionTitle: Item 1.    Financial Statements
NotYetClassifiedElement: Apple Inc.
NotYetClassifiedElement: CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS (Unaudited)
NotYetClassifiedElement: (In millions, except number of ...ousands, and per-share amounts)
TableElement: Table with ~24 rows, ~40 numbers, and 742 characters.
NotYetClassifiedElement: See accompanying Notes to Conde...solidated Financial Statements.
NotYetClassifiedElement: Apple Inc. | Q1 2024 Form 10-Q | 1
NotYetClassifiedElement: Apple Inc.
NotYetClassifiedElement: CONDENSED CONSOLIDATED STATEMEN...OMPREHENSIVE INCOME (Unaudited)
NotYetClassifiedElement: (In millions)
TableElement: Table with ~13 rows, ~22 numbers, and 839 characters.
NotYetClassifiedElement: See accompanying Notes to Conde...solidated Financial Statements.
NotYetClassifiedElement: Apple Inc. | Q1 2024 Form 10-Q | 2
NotYetClassifiedElement: Apple Inc.
NotYetClassifiedElement: CONDENSED CONSOLIDATED B

In [87]:
full_text_output = render_full_text(elements)
print(full_text_output)

PagePart IItem 1.Financial Statements1Item 2.Management’s Discussion and Analysis of Financial Condition and Results of Operations13Item 3.Quantitative and Qualitative Disclosures About Market Risk18Item 4.Controls and Procedures18Part IIItem 1.Legal Proceedings19Item 1A.Risk Factors19Item 2.Unregistered Sales of Equity Securities and Use of Proceeds20Item 3.Defaults Upon Senior Securities21Item 4.Mine Safety Disclosures21Item 5.Other Information21Item 6.Exhibits21
PART I  —  FINANCIAL INFORMATION
Item 1.    Financial Statements
Apple Inc.
CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS (Unaudited)
(In millions, except number of shares, which are reflected in thousands, and per-share amounts)
Three Months EndedDecember 30,2023December 31,2022Net sales:   Products$96,458 $96,388    Services23,117 20,766 Total net sales119,575 117,154 Cost of sales:   Products58,440 60,765    Services6,280 6,057 Total cost of sales64,720 66,822 Gross margin54,855 50,332 Operating expenses:Research and de